In [1]:
import pandas as pd
import os
from Levenshtein import distance

words = ["ameise", "begräbnis", "deichsel", "elster", "fledermaus", "gurke", "hagebutte", "hebamme", "kartoffel", "maulwurf", "pflaume", "stecknadel", "ziege", "zimmerfliege"]

In [2]:
for word in words:
    path = f'/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem/{word}'
    entries = os.listdir(path)

    for entry in entries:
        df = pd.read_csv(f"{path}/{entry}", sep = "\t")
        df = df.drop(columns = ["phontype_maurer", "phontype_DWA", "Levenshtein_phontype", "lextype_maurer", "lextype_DWA", "Levenshtein_lextype", "Unnamed: 0"])
        df = df.drop(df[df["Levenshtein_item"] == 999].index)
        if len(df) == 0:
            continue
        else:
            df_t = df.T
            index = 5000
            for i in df_t:
                if ", " in df_t[i][4] and ", " not in df_t[i][5]:
                    original = df_t[i]
                    df_t.drop(i, axis = 1)
                    words_split = original[4].split(", ")
                    for w in words_split:
                        index += 1
                        row_new = original
                        row_new[4] = w
                        df_t[f"{index}"] = row_new
                elif ", " in df_t[i][4] and ", " in df_t[i][5]:
                    original = df_t[i]
                    df_t.drop(i, axis = 1)
                    wort_1 = df_t[i][4].split(", ")
                    wort_2 = df_t[i][5].split(", ")
                    for eins in wort_1:
                        for zwei in wort_2:
                            index += 1
                            row_new = original
                            row_new[4] = eins
                            row_new[5] = zwei
                            df_t[f"{index}"] = row_new
                elif ", " not in df_t[i][4] and ", " in df_t[i][5]:
                    original = df_t[i]
                    df_t.drop(i, axis = 1)
                    words_split = original[5].split(", ")
                    for w in words_split:
                        index += 1
                        row_new = original
                        row_new[5] = w
                        df_t[f"{index}"] = row_new

            df = df_t.T
            
            results = list(map(lambda x,y: distance(x,y), df["item_maurer"], df["item_DWA"]))
            df["Levenshtein_item"] = results

            df = df.drop(df[df["Levenshtein_item"] == 0].index)
            
            if len(df) == 0:
                continue
            else:
                name = entry.split("lexem_")
                df.to_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem_variance/{word}/variance_{name[1]}", sep = "\t")   

/tmp/ipykernel_6817/4131411144.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_t[f"{index}"] = row_new
/tmp/ipykernel_6817/4131411144.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_t[f"{index}"] = row_new
/tmp/ipykernel_6817/4131411144.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy(

In [3]:
for word in words:
    path = f'/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem_variance/{word}'
    entries = os.listdir(path)

    for entry in entries:
        df = pd.read_csv(f"{path}/{entry}", sep = "\t")
        df = df.drop(columns = ["Unnamed: 0"])

        spaces_maurer = []
        for item in df["item_maurer"]:
            space = " ".join(i for i in item)
            space = space.replace("  ", " ")

            index = 0
            for j in space[:-2]:
                if j == space[index+2] and j != " ":
                    a = space[index]
                    b = space[index+2]
                    space = space.replace(f"{a} {b}", f"{a}{b}")
                    break
                index +=1

            space = space.replace("( ", "(")
            space = space.replace(" )", ")")
            space = space.replace("c k", "ck")
            space = space.replace("s c h", "sch")
            space = space.replace("c h", "ch")
            space = space.replace("i e", "ie")
            space = space.replace("e i", "ei")
            space = space.replace("i (e)", "i(e)")
            space = space.replace("t z", "tz")

            spaces_maurer.append(space)

        df.insert(loc = 5, column = "item_maurer_align", value = spaces_maurer)

        spaces_dwa = []
        for item in df["item_DWA"]:
            space = " ".join(i for i in item)
            space = space.replace("  ", " ")

            index = 0
            for j in space[:-2]:
                if j == space[index+2] and j != " ":
                    a = space[index]
                    b = space[index+2]
                    space = space.replace(f"{a} {b}", f"{a}{b}")
                    break
                index +=1

            space = space.replace("( ", "(")
            space = space.replace(" )", ")")
            space = space.replace("c k", "ck")
            space = space.replace("s c h", "sch")
            space = space.replace("c h", "ch")
            space = space.replace("i e", "ie")
            space = space.replace("e i", "ei")
            space = space.replace("i (e)", "i(e)")
            space = space.replace("t z", "tz")

            spaces_dwa.append(space)

        df.insert(loc = 7, column = "item_DWA_align", value = spaces_dwa)
        df.to_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem_variance/{word}/{entry}", sep = "\t") 
        